In [1]:
import pandas as pd
import glob
import json

# For Location Extraction
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

In [2]:
# For Extracting Location from Text

# Loading the Stanford jar file
import os
java_path = "C:/Program Files/Java/jdk-14.0.2/bin/java.exe"
os.environ['JAVAHOME'] = java_path

st = StanfordNERTagger('./stanford-ner-4.0.0/classifiers/english.all.3class.distsim.crf.ser.gz', './stanford-ner-4.0.0/stanford-ner.jar', encoding='utf-8')
 
def location_from_text(text):
    print(text)
    locations = ''
    temp = [ i.capitalize() for i in text.split(' ')]
    text = ''
    for i in temp:
        text += (i + ' ')
        
    text = text[:-1]

    tokenized_text = word_tokenize(text)
    classified_text = st.tag(tokenized_text)
    
    for i in classified_text:
        if i[1] == 'LOCATION':
            locations += i[0] + ","
            
    return locations[:-1]

In [3]:
folders = glob.glob("Data/*")

In [4]:
dictlist = []

for folder in folders:
    if folder == "Data/user":
        continue
    files = glob.glob(folder + "/tweets/*")
    for file in files:
        json_string = open(file,'r',encoding="utf8").read()
        json_dict = json.loads(json_string)
        json_dict["search_term"] = folder[5:].capitalize()
        dictlist.append(json_dict)

In [5]:
df = pd.DataFrame(dictlist)
df = df.replace({'\n': ' '}, regex=True) # remove linebreaks in the dataframe
df = df.replace({'\t': ' '}, regex=True) # remove tabs in the dataframe
df = df.replace({'\r': ' '}, regex=True) # remove carriage return in the dataframe

In [6]:
df.sort_values("datetime",ascending = True,inplace=True,na_position="last")

In [7]:
df

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet,user_id,search_term,has_media,medias
2343,wikirandom,12870169175,Early thermal weapons http://en.wikipedia.org...,/wikirandom/status/12870169175,0,0,0,2010-04-26 13:00:04,False,False,124412057,early thermal weapons,NaN,NaN
2347,wiki_bot,42592438874943488,http://bit.ly/fBN9zg Early thermal weapons #...,/wiki_bot/status/42592438874943488,0,0,0,2011-03-01 20:00:03,False,False,122383622,early thermal weapons,NaN,NaN
3724,BeerBanzai,66836933933928448,(CONTD. FROM PREVIOUS) → Therefore we must str...,/BeerBanzai/status/66836933933928448,0,0,0,2011-05-07 17:39:01,False,False,92426796,interplate earthquake,NaN,NaN
2352,AlmanaraMedia,79645490265866240,@Sarahdrah this is about the early thermal we...,/AlmanaraMedia/status/79645490265866240,0,0,1,2011-06-12 01:55:39,True,False,99856628,early thermal weapons,NaN,NaN
2344,CluG1rleso,157775084457701376,"Early thermal weapons : Mining (military), Sie...",/CluG1rleso/status/157775084457701376,0,0,0,2012-01-13 16:14:46,False,False,430831584,early thermal weapons,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399,eq_map_ww2,1290611335395528705,"【M4.7】NEAR EAST COAST OF HONSHU, JAPAN Depth 8...",/eq_map_ww2/status/1290611335395528705,0,0,0,2020-08-04 17:01:35,False,False,1144740888,earthquakes,True,[https://t.co/chXLWuXpWF]
3437,triplejplays,1290611335248531458,.@HockeyDadband - Heavy Assault [21:31],/triplejplays/status/1290611335248531458,0,0,0,2020-08-04 17:01:35,False,False,86848460,heavy,NaN,NaN
214,JstLangevin,1290611335970148352,Mais nos autorites n’ont pas l’ air de s’en r...,/JstLangevin/status/1290611335970148352,0,0,0,2020-08-04 17:01:35,False,False,1241669749,air,NaN,NaN
3765,jtnpfNIl0QSLfKM,1290611334392852480,入れさせてもらった…(ニッコリ),/jtnpfNIl0QSLfKM/status/1290611334392852480,0,0,0,2020-08-04 17:01:35,True,False,1290063380019998721,jupiter,NaN,NaN


In [8]:
df["location_from_text"] = df["text"].apply(location_from_text , 1)
df

UnboundLocalError: local variable 'i' referenced before assignment

In [ ]:
with open("CollectedData.csv" , "w", encoding="utf-8") as f:
    df.to_csv(f,index=False)

In [ ]:
import requests , bs4,urllib
df = pd.DataFrame(pd.read_csv("CollectedData.csv"))
image_df = df[["ID","has_media","medias","search_term"]]
image_df.dropna(inplace=True)
indexes = image_df[image_df["medias"] == "[]"].index
image_df.drop(indexes,inplace=True)
image_df.head()

In [18]:
import mimetypes,os,os.path

def is_url_image(url):    
    mimetype,encoding = mimetypes.guess_type(url)
    print((mimetype and mimetype.startswith('image')))
    return (mimetype and mimetype.startswith('image'))

In [19]:
def download_image(item):
    print(item)
    import requests
    import time
    if item["medias"] == None:
        return 
    item["medias"] = item["medias"].strip("][")
    item["medias"] = item["medias"].strip("\'")
    
    r = requests.get(item["medias"], stream=True)
    if os.path.isdir("Data/{}/content".format(item["search_term"])) == False:
        os.mkdir("Data/{}/content".format(item["search_term"]))
    ext = r.headers['content-type'].split('/')[-1]
    ext = ext.split(";")[0] # converts response headers mime type to an extension (may not work with everything)
    with open("Data/{}/content/{}.{}".format(item["search_term"],item["ID"],ext), 'wb') as f: # open the file to write as binary - replace 'wb' with 'w' for text files
        for chunk in r.iter_content(1024): # iterate on stream using 1KB packets
            f.write(chunk) # write the file


image_df = image_df.apply(download_image , 1)

ID                     1010193501798191104
has_media                             True
medias         ['https://t.co/CyP4LtCbiP']
search_term                   zipingpu dam
Name: 94, dtype: object
ID                     1016774079611797504
has_media                             True
medias         ['https://t.co/BA8THZOQwG']
search_term                   zipingpu dam
Name: 97, dtype: object
ID                     1060656032219840512
has_media                             True
medias         ['https://t.co/b1E3yOprAO']
search_term      yellowstone fires of 1988
Name: 104, dtype: object
ID                     1075261237246783488
has_media                             True
medias         ['https://t.co/DFSXZBSBNg']
search_term      yellowstone fires of 1988
Name: 109, dtype: object
ID                     1081830923023368192
has_media                             True
medias         ['https://t.co/gA1izMnxV0']
search_term             ocean surface wave
Name: 115, dtype: object
ID               

KeyboardInterrupt: 

In [ ]:
# For Extracting Location from 